In [4]:
ls

Cluster_index_mat/  Features/  label.mat


In [29]:
ls Features/

s_100_feature.npy  s_213_feature.npy  s_326_feature.npy  s_439_feature.npy
s_101_feature.npy  s_214_feature.npy  s_327_feature.npy  s_43_feature.npy
s_102_feature.npy  s_215_feature.npy  s_328_feature.npy  s_440_feature.npy
s_103_feature.npy  s_216_feature.npy  s_329_feature.npy  s_441_feature.npy
s_104_feature.npy  s_217_feature.npy  s_32_feature.npy   s_442_feature.npy
s_105_feature.npy  s_218_feature.npy  s_330_feature.npy  s_443_feature.npy
s_106_feature.npy  s_219_feature.npy  s_331_feature.npy  s_444_feature.npy
s_107_feature.npy  s_21_feature.npy   s_332_feature.npy  s_445_feature.npy
s_108_feature.npy  s_220_feature.npy  s_333_feature.npy  s_446_feature.npy
s_109_feature.npy  s_221_feature.npy  s_334_feature.npy  s_447_feature.npy
s_10_feature.npy   s_222_feature.npy  s_335_feature.npy  s_448_feature.npy
s_110_feature.npy  s_223_feature.npy  s_336_feature.npy  s_449_feature.npy
s_111_feature.npy  s_224_feature.npy  s_337_feature.npy  s_44_feature.npy
s_112_feature.npy  s_225_fe

Cluster_index_mat/  Features/  label.mat


In [87]:
data=np.load("/content/Dataset_shuai/Cluster_index_mat/s_10_cluster_index.npy",allow_pickle=True)

In [92]:
data.item()['cluster_index_mat'].shape

(45, 54, 45)

In [75]:
import scipy.io as sc
data=sc.loadmat("/content/Dataset_shuai/label.mat")

In [84]:
data['label'].dtype

dtype('uint8')

In [70]:
#feature
data.item()['feature_mat'].shape

(400, 1632)

In [86]:
import numpy as np
data=np.load("s_100_cluster_index.npy",allow_pickle=True)

FileNotFoundError: [Errno 2] No such file or directory: 's_100_cluster_index.npy'

In [5]:
!lscpu

Architecture:                x86_64
  CPU op-mode(s):            32-bit, 64-bit
  Address sizes:             46 bits physical, 48 bits virtual
  Byte Order:                Little Endian
CPU(s):                      2
  On-line CPU(s) list:       0,1
Vendor ID:                   GenuineIntel
  Model name:                Intel(R) Xeon(R) CPU @ 2.20GHz
    CPU family:              6
    Model:                   79
    Thread(s) per core:      2
    Core(s) per socket:      1
    Socket(s):               1
    Stepping:                0
    BogoMIPS:                4399.99
    Flags:                   fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pg
                             e mca cmov pat pse36 clflush mmx fxsr sse sse2 ss h
                             t syscall nx pdpe1gb rdtscp lm constant_tsc rep_goo
                             d nopl xtopology nonstop_tsc cpuid tsc_known_freq p
                             ni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2ap
                   

In [39]:
data.item()['feature_mat'].shape

(400, 1632)

In [ ]:
# brain_dataset.py
"""
Custom PyTorch Dataset and DataLoaders for the neuroimaging toy dataset.
Assumptions:
- label.mat contains 'label' variable (shape: 500 or 500×1)
- Feature files: s1_feature.mat, s2_feature.mat, ..., s500_feature.mat
  Inside: 'F' → (400, 1632) float array
- Cluster files: s1_cluster_index.mat, s2_cluster_index.mat, ..., s500_cluster_index.mat
  Inside: 'C' → (45, 54, 45) integer array
"""

import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import scipy.io as sio
from sklearn.model_selection import train_test_split
from typing import Optional, Dict, Any
# import train_config


class BrainROIDataset(Dataset):
    """
    Loads per-subject ROI feature matrices (400×1632) and optionally cluster maps.
    Labels are loaded once from label.mat and indexed by subject number (1-based).
    """
    def __init__(
        self,
        subject_ids: list[int],
        label_data_dir: str,
        feature_data_dir: str,
        cluster_data_dir: str,
        load_cluster: bool = False,
        feature_key: str = 'F',
        cluster_key: str = 'C'
    ):
        """
        Args:
            subject_ids: List of subject IDs (integers 1 to 500)
            data_dir: Root directory containing all .mat files
            load_cluster: Whether to load the 3D cluster maps (memory intensive)
            feature_key: Key name for feature matrix in .mat file
            cluster_key: Key name for cluster index in .mat file
        """
        self.subject_ids = sorted(subject_ids)
        self.data_dir = label_data_dir
        self.feature_data=feature_data_dir
        self.cluster_data=cluster_data_dir
        self.load_cluster = load_cluster
        self.feature_key = feature_key
        self.cluster_key = cluster_key

        # Load global labels once
        label_path = self.data_dir
        if not os.path.exists(label_path):
            raise FileNotFoundError(f"Label file not found: {label_path}")

        labels_mat = sio.loadmat(label_path)
        self.labels = labels_mat['label'].flatten().astype(np.int64)  # shape (500,)

        if len(self.labels) != 500:
            raise ValueError(f"Expected 500 labels, got {len(self.labels)}")

    def __len__(self) -> int:
        return len(self.subject_ids)

    def __getitem__(self, idx: int) -> Dict[str, Any]:
        sid = self.subject_ids[idx]  # 1-based subject ID

        # 1. Load feature matrix
        feature_path = os.path.join(self.feature_data, f"s_{sid}_feature.npy")
        if not os.path.exists(feature_path):
            raise FileNotFoundError(f"Feature file missing: {feature_path}")

        feat_data = np.load(feature_path)
        features=feat_data.item()['feature_mat']
        # features = feat_data[self.feature_key].astype(np.float32)  # (400, 1632)
        
        cluster_path=os.path.join(self.cluster_data, f"s_{sid}_cluster_index.npy")
        if not os.path.exists(cluster_path):
            raise FileNotFoundError(f"Cluster file missing: {cluster_path}")
        clus_data=np.load(cluster_path)
        clusters_matrix=clus_data.item()['cluster_index_mat']

        # 2. Optional: Load cluster map
        # cluster = None
        # if self.load_cluster:
        #     cluster_path = os.path.join(self.data_dir, f"s{sid}_cluster_index.mat")
        #     if not os.path.exists(cluster_path):
        #         raise FileNotFoundError(f"Cluster file missing: {cluster_path}")

        #     clust_data = sio.loadmat(cluster_path)
        #     cluster = clust_data[self.cluster_key].astype(np.int16)  # (45,54,45)

        # 3. Get corresponding label (0-based index in array)
        label = self.labels[sid - 1]

        # Prepare output
        item = {
            'features': torch.from_numpy(features),     # [400, 1632]
            'label': torch.tensor(label, dtype=torch.long),
            'subject_id': sid
        }

        # if cluster is not None:
        #     item['cluster_map'] = torch.from_numpy(cluster)

        return item


def create_dataloaders(
    label_data_dir,
    feature_data_dir,
    cluster_data_dir,
    batch_size: int = 16,
    num_workers: int = 4,
    pin_memory: bool = True,
    load_cluster: bool = False,
    val_size: float = 0.15,
    test_size: float = 0.15,
    random_state: int = 42
) -> tuple[DataLoader, DataLoader, DataLoader]:
    """
    Creates train/val/test DataLoaders with standard split (70/15/15 by default)

    Returns:
        (train_loader, val_loader, test_loader)
    """
    all_subjects = list(range(1, 501))  # 1 to 500

    # Stratified split (though binary labels, still good practice)
    train_subjs, temp_subjs = train_test_split(
        all_subjects,
        test_size=(val_size + test_size),
        random_state=random_state
    )

    val_subjs, test_subjs = train_test_split(
        temp_subjs,
        test_size=test_size / (val_size + test_size),
        random_state=random_state
    )

    print(f"Dataset split → Train: {len(train_subjs)} | Val: {len(val_subjs)} | Test: {len(test_subjs)}")

    train_ds = BrainROIDataset(train_subjs, label_data_dir, feature_data_dir, cluster_data_dir, load_cluster=load_cluster)
    val_ds   = BrainROIDataset(val_subjs,   label_data_dir, feature_data_dir, cluster_data_dir, load_cluster=load_cluster)
    test_ds  = BrainROIDataset(test_subjs,  label_data_dir, feature_data_dir, cluster_data_dir, load_cluster=load_cluster)

    train_loader = DataLoader(
        train_ds,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=pin_memory,
        drop_last=True
    )

    val_loader = DataLoader(
        val_ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=pin_memory
    )

    test_loader = DataLoader(
        test_ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=pin_memory
    )

    return train_loader, val_loader, test_loader


# ──────────────────────────────────────────────────────────────────────────────
# Quick test / example usage
# ──────────────────────────────────────────────────────────────────────────────



In [ ]:
/content/Dataset_shuai/Cluster_index_mat/
/content/Dataset_shuai/Features/
/content/Dataset_shuai/label.mat

In [46]:
ls

Cluster_index_mat/  Features/  label.mat


In [12]:

# Change this to your actual data location
DATA_ROOT = "/content/Dataset_shuai/"   # ← IMPORTANT: update this path

train_loader, val_loader, test_loader = create_dataloaders(
    label_data_dir="/content/Dataset_shuai/",
    feature_data_dir="/content_Dataset_shuai/Features",
    cluster_data_dir="/content/Dataset_shuai/Cluster_index_mat",
    batch_size=8,
    load_cluster=False,       # ← Set True only if you really need it
)

# Get one batch to verify
batch = next(iter(train_loader))

print("\nBatch shapes:")
print(f"features    : {batch['features'].shape}")     # [batch, 400, 1632]
print(f"labels      : {batch['label'].shape}")        # [batch]
print(f"subject_ids : {batch['subject_id']}")

if 'cluster_map' in batch:
    print(f"cluster_map : {batch['cluster_map'].shape}")

    

TypeError: create_dataloaders() got an unexpected keyword argument 'label_data_dir'

In [6]:
from google.colab import drive
drive.mount('/content/drive')

ValueError: mount failed

In [9]:
!wget -O "Dataset_shuai" "https://drive.google.com/drive/u/0/folders/1FqNaf3EHFwARjTg6d14OZBzM3h-2Rtts"

--2026-01-10 06:22:37--  https://drive.google.com/drive/u/0/folders/1FqNaf3EHFwARjTg6d14OZBzM3h-2Rtts
Resolving drive.google.com (drive.google.com)... 173.194.206.113, 173.194.206.100, 173.194.206.138, ...
Connecting to drive.google.com (drive.google.com)|173.194.206.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://drive.google.com/drive/folders/1FqNaf3EHFwARjTg6d14OZBzM3h-2Rtts [following]
--2026-01-10 06:22:37--  https://drive.google.com/drive/folders/1FqNaf3EHFwARjTg6d14OZBzM3h-2Rtts
Reusing existing connection to drive.google.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://accounts.google.com/ServiceLogin?service=wise&passive=1209600&osid=1&continue=https://drive.google.com/drive/folders/1FqNaf3EHFwARjTg6d14OZBzM3h-2Rtts&followup=https://drive.google.com/drive/folders/1FqNaf3EHFwARjTg6d14OZBzM3h-2Rtts [following]
--2026-01-10 06:22:37--  https://accounts.google.com/ServiceLogin?service=wise&passive=1209600&osid=

In [34]:
!gdown --folder --remaining-ok 1FqNaf3EHFwARjTg6d14OZBzM3h-2Rtts

Retrieving folder contents
Retrieving folder 1sc-h3XY9AMfvilcrTvLZGGpjsWz9WIZD Cluster_index_mat
Processing file 1sQ5dCQ49QitQkm_i9o83xom9EyA6ykvZ s_1_cluster_index.npy
Processing file 1wPAP0flGNS7FJyRaWeILVWqXpGW_SyOB s_2_cluster_index.npy
Processing file 1K8IWwxZeSk5UfdKwNdH-6kzitUW3KlTR s_3_cluster_index.npy
Processing file 1F0gmp_ugnAqHVWI88XSqgy103GDZCzW4 s_4_cluster_index.npy
Processing file 1ZdHhIzJ8tU2nXnHC6hcgjaOPsJogsgEb s_5_cluster_index.npy
Processing file 1QHDt-RlW7SiYpydbmXxWNECwEKQ0XBja s_6_cluster_index.npy
Processing file 1e-zFqTrwd1CfGzxhWY2NxEFG0tGGUms2 s_7_cluster_index.npy
Processing file 1HBpJWTQfc7uzidUlKhvM_-depOaVPi6T s_8_cluster_index.npy
Processing file 1KnOufqfZX3E5eVGeE_LDBET_zq7qS81k s_9_cluster_index.npy
Processing file 1XYxySfUy_FQnM6-ba1ckI2YaNVyWpc3c s_10_cluster_index.npy
Processing file 1BiuqExb7QIDLmlDCgkgu2oQniU8K_Wla s_11_cluster_index.npy
Processing file 1BlrBpLqzE3hCWXdYPbAwW_vywxEYlOKk s_12_cluster_index.npy
Processing file 1LCw24O7WM-ehRaaR6sf

In [35]:
import os

In [32]:
!rm Dataset_shuai

In [27]:
os.listdir('/content/Dataset_shuai')

NotADirectoryError: [Errno 20] Not a directory: '/content/Dataset_shuai'

In [33]:
ls

sample_data/
